In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..', '..', '..', '..'))
sys.path.insert(0, module_path)

### Подготовка
1. Загрузим данные
2. Поделим их на train и test

In [4]:

from brs.models.rate_prediction.baseline.model import BaselineRateModel
from brs.ingestion import LoadLocalDataset

# Подгружаем сырые данные
lld = LoadLocalDataset()
data_books = lld.load_dataset("books")
data_users = lld.load_dataset("users")
data_ratings = lld.load_dataset("ratings")

# Делим на train и test
num_ratings = data_ratings.shape[0]
split = round(num_ratings * 0.9)
train, test = data_ratings[:split], data_ratings[split:]

d:\z\Conspects\_Summer\BRS\brs\ingestion\load_local.py:42: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(dataset_path)


### Pipeline/preprocess

- Считаем среднюю оценку для каждой книги на train
- Предсказываем каждому пользователю на test именно её

In [5]:
from brs.models.rate_prediction.baseline.pipeline import BaselinePipeline

pipeline = BaselinePipeline()
pipeline.fit(data_users, train, data_books)
train = pipeline.transform(train)
test = pipeline.transform(test)

model = BaselineRateModel()
model.fit(train)

predictions = model.predict(test)


### Считаем качество наших предсказаний
У нас три ключевых метрики
- MAE
- MSE
- RMSE

Зафиксируем их, как качество самой простой модели

In [6]:
from brs.metrics.regression_metrics import MAE, RMSE, MSE

mae_score = MAE().estimate(predictions, test["Book-Rating"])
rmse_score = RMSE().estimate(predictions, test["Book-Rating"])
mse_score = MSE().estimate(predictions, test["Book-Rating"])

print(f"MAE: {mae_score}")
print(f"RMSE: {rmse_score}")
print(f"MSE: {mse_score}")

MAE: 3.743918537609926
RMSE: 4.384962666398486
MSE: 19.227897585708522
